---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

# Assignment 2 - Pandas Introduction
All questions are weighted the same in this assignment.
## Part 1
The following code loads the olympics dataset (olympics.csv), which was derrived from the Wikipedia entry on [All Time Olympic Games Medals](https://en.wikipedia.org/wiki/All-time_Olympic_Games_medal_table), and does some basic data cleaning. 

The columns are organized as # of Summer games, Summer medals, # of Winter games, Winter medals, total # number of games, total # of medals. Use this dataset to answer the questions below.

In [2]:
import pandas as pd

df = pd.read_csv('olympics.csv', index_col=0, skiprows=1)

for col in df.columns:
    if col[:2]=='01':
        df.rename(columns={col:'Gold'+col[4:]}, inplace=True)
    if col[:2]=='02':
        df.rename(columns={col:'Silver'+col[4:]}, inplace=True)
    if col[:2]=='03':
        df.rename(columns={col:'Bronze'+col[4:]}, inplace=True)
    if col[:1]=='№':
        df.rename(columns={col:'#'+col[1:]}, inplace=True)

names_ids = df.index.str.split('\s\(') # split the index by '('

df.index = names_ids.str[0] # the [0] element is the country name (new index) 
df['ID'] = names_ids.str[1].str[:3] # the [1] element is the abbreviation or ID (take first 3 characters from that)

df = df.drop('Totals')
df.head()

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
Afghanistan,13,0,0,2,2,0,0,0,0,0,13,0,0,2,2,AFG
Algeria,12,5,2,8,15,3,0,0,0,0,15,5,2,8,15,ALG
Argentina,23,18,24,28,70,18,0,0,0,0,41,18,24,28,70,ARG
Armenia,5,1,2,9,12,6,0,0,0,0,11,1,2,9,12,ARM
Australasia,2,3,4,5,12,0,0,0,0,0,2,3,4,5,12,ANZ


### Question 0 (Example)

What is the first country in df?

*This function should return a Series.*

In [2]:
# You should write your whole answer within the function provided. The autograder will call
# this function and compare the return value against the correct solution value
def answer_zero():
    # This function returns the row for Afghanistan, which is a Series object. The assignment
    # question description will tell you the general format the autograder is expecting
    return df.iloc[0]

# You can examine what your function returns by calling it in the cell. If you have questions
# about the assignment formats, check out the discussion forums for any FAQs
answer_zero() 

# Summer           13
Gold                0
Silver              0
Bronze              2
Total               2
# Winter            0
Gold.1              0
Silver.1            0
Bronze.1            0
Total.1             0
# Games            13
Gold.2              0
Silver.2            0
Bronze.2            2
Combined total      2
ID                AFG
Name: Afghanistan, dtype: object

### Question 1
Which country has won the most gold medals in summer games?

*This function should return a single string value.*

In [3]:
def answer_one():
    return df['Gold'].idxmax(axis=1)

answer_one()

'United States'

### Question 2
Which country had the biggest difference between their summer and winter gold medal counts?

*This function should return a single string value.*

In [4]:
def answer_two():
    df['DiferenceGold'] = abs(df['Gold'] - df['Gold.1'])
    return df['DiferenceGold'].idxmax(axis=1)
answer_two()

'United States'

### Question 3
Which country has the biggest difference between their summer gold medal counts and winter gold medal counts relative to their total gold medal count? 

$$\frac{Summer~Gold - Winter~Gold}{Total~Gold}$$

Only include countries that have won at least 1 gold in both summer and winter.

*This function should return a single string value.*

In [5]:
def answer_three():
    df['RelativeDiferenceGold'] = 0.000000000   
    for x in range(len(df['RelativeDiferenceGold'])):
        if (df['Gold'][x]>0 and df['Gold.1'][x]>0):
            valor = abs(df['Gold'][x] - df['Gold.1'][x])/df['Gold.2'][x]
            df['RelativeDiferenceGold'][x] = valor
    return df['RelativeDiferenceGold'].idxmax(axis=1)
answer_three()

'Bulgaria'

### Question 4
Write a function that creates a Series called "Points" which is a weighted value where each gold medal (`Gold.2`) counts for 3 points, silver medals (`Silver.2`) for 2 points, and bronze medals (`Bronze.2`) for 1 point. The function should return only the column (a Series object) which you created, with the country names as indices.

*This function should return a Series named `Points` of length 146*

In [6]:
def answer_four():
    df['Points'] = 3*df['Gold.2']+2*df['Silver.2'] + 1*df['Bronze.2']
    return df['Points']
answer_four()

Afghanistan                            2
Algeria                               27
Argentina                            130
Armenia                               16
Australasia                           22
Australia                            923
Austria                              569
Azerbaijan                            43
Bahamas                               24
Bahrain                                1
Barbados                               1
Belarus                              154
Belgium                              276
Bermuda                                1
Bohemia                                5
Botswana                               2
Brazil                               184
British West Indies                    2
Bulgaria                             411
Burundi                                3
Cameroon                              12
Canada                               846
Chile                                 24
China                               1120
Colombia        

## Part 2
For the next set of questions, we will be using census data from the [United States Census Bureau](http://www.census.gov). Counties are political and geographic subdivisions of states in the United States. This dataset contains population data for counties and states in the US from 2010 to 2015. [See this document](https://www2.census.gov/programs-surveys/popest/technical-documentation/file-layouts/2010-2015/co-est2015-alldata.pdf) for a description of the variable names.

The census dataset (census.csv) should be loaded as census_df. Answer questions using this as appropriate.

### Question 5
Which state has the most counties in it? (hint: consider the sumlevel key carefully! You'll need this for future questions too...)

*This function should return a single string value.*

In [3]:
census_df = pd.read_csv('census.csv')
census_df.head()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
0,40,3,6,1,0,Alabama,Alabama,4779736,4780127,4785161,...,0.002295,-0.193196,0.381066,0.582002,-0.467369,1.030015,0.826644,1.383282,1.724718,0.712594
1,50,3,6,1,1,Alabama,Autauga County,54571,54571,54660,...,7.242091,-2.915927,-3.012349,2.265971,-2.530799,7.606016,-2.626146,-2.722002,2.592270,-2.187333
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183193,...,14.832960,17.647293,21.845705,19.243287,17.197872,15.844176,18.559627,22.727626,20.317142,18.293499
3,50,3,6,1,5,Alabama,Barbour County,27457,27457,27341,...,-4.728132,-2.500690,-7.056824,-3.904217,-10.543299,-4.874741,-2.758113,-7.167664,-3.978583,-10.543299
4,50,3,6,1,7,Alabama,Bibb County,22915,22919,22861,...,-5.527043,-5.068871,-6.201001,-0.177537,0.177258,-5.088389,-4.363636,-5.403729,0.754533,1.107861


In [8]:
def answer_five():
    #census_df_2 = census_df.set_index(['STNAME', 'STATE'])
    census_df_2 = census_df['STNAME']
    dataframe = pd.Series({'State':0})
    for x in range(len(census_df_2)):
        dataframe[census_df_2[x]] = 0
    for x in range(len(census_df_2)):
        dataframe[census_df_2[x]] = dataframe[census_df_2[x]] + 1
    #print(dataframe)
    return dataframe.idxmax()
answer_five()

'Texas'

### Question 6
**Only looking at the three most populous counties for each state**, what are the three most populous states (in order of highest population to lowest population)? Use `CENSUS2010POP`.

*This function should return a list of string values.*

In [60]:
def answer_six():
    census_df_3 = census_df[census_df['SUMLEV']==50]
    columns_to_keep =  ['STNAME', 'CENSUS2010POP']
    census_df_3 = census_df_3[columns_to_keep]
    census_df_4 = census_df_3.set_index('STNAME')
    cdf = pd.Series()
    cdf_2 = pd.Series()
    for x in range(len(census_df_3.index)):
        cdf[census_df_4.index[x]] = [0,0,0]
    for i in cdf.index:
        df_aux = census_df_3[census_df_3['STNAME']==i]
        df_aux = df_aux.sort_values(['CENSUS2010POP'])
        print(df_aux)
        lista = [x+1 for x in range(len(df_aux))]
        #print(lista)
        df_aux=df_aux.reset_index(lista)
        print(df_aux)
        if len(df_aux)>=3:
            for j in range(0,3):

                cdf[i][j] = df_aux['CENSUS2010POP'].max()
                idmax = df_aux['CENSUS2010POP'].idxmax()
                df_aux = df_aux.drop(idmax)
        else:
            for j in range(0,len(df_aux)):
                cdf[i][j] = df_aux['CENSUS2010POP'].max()
                idmax = df_aux['CENSUS2010POP'].idxmax()
                df_aux = df_aux.drop(idmax)
            
    for i in cdf.index:
        cdf_2[i] = 0
        for j in range(0,3):
            cdf_2[i] += cdf[i][j]
    #print(cdf_2)
    
    result = []
    for i in range(0,3):
        idmax = cdf_2.idxmax()
        #print(idmax)
        cdf_2 = cdf_2.drop(idmax)
        result.append(idmax)
    #print(result)
    return result
answer_six()

     STNAME  CENSUS2010POP
32  Alabama           9045
53  Alabama          10591
6   Alabama          10914
43  Alabama          11299
19  Alabama          11539
66  Alabama          11670
18  Alabama          13228
60  Alabama          13763
12  Alabama          13859
21  Alabama          13906
14  Alabama          13932
38  Alabama          14564
15  Alabama          14972
33  Alabama          15760
29  Alabama          17241
34  Alabama          17302
65  Alabama          17581
54  Alabama          19746
7   Alabama          20947
46  Alabama          21027
44  Alabama          21452
56  Alabama          22913
4   Alabama          22915
50  Alabama          23068
67  Alabama          24484
13  Alabama          25833
10  Alabama          25989
31  Alabama          26790
3   Alabama          27457
47  Alabama          30776
..      ...            ...
11  Alabama          43643
24  Alabama          43820
16  Alabama          49948
23  Alabama          50251
57  Alabama          52947
3

      STNAME  CENSUS2010POP
528  Georgia           1717
515  Georgia           2513
549  Georgia           2799
459  Georgia           3082
427  Georgia           3183
401  Georgia           3451
447  Georgia           4034
520  Georgia           5010
546  Georgia           5834
525  Georgia           6058
497  Georgia           6125
416  Georgia           6694
429  Georgia           6798
527  Georgia           6865
537  Georgia           6885
550  Georgia           7421
517  Georgia           7719
487  Georgia           7996
479  Georgia           8340
399  Georgia           8375
522  Georgia           8729
495  Georgia           8742
530  Georgia           8906
539  Georgia           8930
540  Georgia           9023
500  Georgia           9123
553  Georgia           9255
532  Georgia           9315
467  Georgia           9429
474  Georgia           9538
..       ...            ...
460  Georgia          79626
544  Georgia          83768
519  Georgia          85215
444  Georgia        

    index     STNAME  CENSUS2010POP
0    1185  Louisiana           5252
1    1143  Louisiana           6839
2    1149  Louisiana           7759
3    1172  Louisiana           9091
4    1142  Louisiana          10132
5    1144  Louisiana          10407
6    1177  Louisiana          11203
7    1193  Louisiana          11604
8    1164  Louisiana          12093
9    1138  Louisiana          14353
10   1161  Louisiana          14890
11   1195  Louisiana          15313
12   1194  Louisiana          15625
13   1156  Louisiana          16274
14   1145  Louisiana          17195
15   1150  Louisiana          20267
16   1173  Louisiana          20725
17   1152  Louisiana          20767
18   1146  Louisiana          20822
19   1178  Louisiana          22102
20   1153  Louisiana          22309
21   1187  Louisiana          22721
22   1170  Louisiana          22802
23   1169  Louisiana          23042
24   1135  Louisiana          23421
25   1192  Louisiana          23788
26   1174  Louisiana        

        STNAME  CENSUS2010POP
1684  Nebraska            460
1686  Nebraska            478
1740  Nebraska            539
1719  Nebraska            614
1739  Nebraska            632
1767  Nebraska            647
1685  Nebraska            690
1727  Nebraska            736
1738  Nebraska            763
1773  Nebraska            818
1733  Nebraska            824
1724  Nebraska            967
1764  Nebraska           1311
1756  Nebraska           1526
1706  Nebraska           1941
1710  Nebraska           2008
1718  Nebraska           2044
1717  Nebraska           2049
1716  Nebraska           2057
1689  Nebraska           2099
1720  Nebraska           2538
1713  Nebraska           2756
1748  Nebraska           2773
1725  Nebraska           2908
1749  Nebraska           2970
1690  Nebraska           3145
1763  Nebraska           3152
1712  Nebraska           3225
1723  Nebraska           3423
1744  Nebraska           3735
...        ...            ...
1704  Nebraska           9182
1750  Nebr

      STNAME  CENSUS2010POP
2280  Oregon           1441
2273  Oregon           1765
2256  Oregon           1871
2277  Oregon           7008
2258  Oregon           7422
2257  Oregon           7445
2264  Oregon           7895
2270  Oregon          11173
2246  Oregon          16134
2252  Oregon          20978
2261  Oregon          21720
2259  Oregon          22346
2253  Oregon          22364
2278  Oregon          25213
2274  Oregon          25250
2276  Oregon          25748
2268  Oregon          31313
2249  Oregon          37039
2266  Oregon          46034
2250  Oregon          49351
2251  Oregon          63043
2263  Oregon          66380
2272  Oregon          75403
2275  Oregon          75889
2262  Oregon          82713
2247  Oregon          85579
2281  Oregon          99193
2255  Oregon         107667
2267  Oregon         116672
2254  Oregon         157733
2260  Oregon         203206
2269  Oregon         315335
2265  Oregon         351715
2248  Oregon         375992
2279  Oregon        

        STNAME  CENSUS2010POP
2911  Virginia           2321
2986  Virginia           3958
2875  Virginia           4731
2889  Virginia           5190
2970  Virginia           5927
2968  Virginia           5961
2964  Virginia           6650
2877  Virginia           6824
2914  Virginia           6945
2975  Virginia           7042
2979  Virginia           7042
2953  Virginia           7058
2885  Virginia           7256
2941  Virginia           7373
2973  Virginia           8582
2923  Virginia           8978
2942  Virginia           9254
2891  Virginia          10052
2925  Virginia          10959
2894  Virginia          11151
2917  Virginia          11391
2954  Virginia          12087
2988  Virginia          12150
2906  Virginia          12243
2930  Virginia          12330
2972  Virginia          12332
2929  Virginia          12389
2886  Virginia          12586
2870  Virginia          12690
2921  Virginia          12914
...        ...            ...
2896  Virginia          65203
2961  Virg

['California', 'Texas', 'Illinois']

### Question 7
Which county has had the largest absolute change in population within the period 2010-2015? (Hint: population values are stored in columns POPESTIMATE2010 through POPESTIMATE2015, you need to consider all six columns.)

e.g. If County Population in the 5 year period is 100, 120, 80, 105, 100, 130, then its largest change in the period would be |130-80| = 50.

*This function should return a single string value.*

In [50]:
def answer_seven():
    census_df_5 = census_df[census_df['SUMLEV']==50]
    columns_to_keep =  ['STNAME', 'CTYNAME','POPESTIMATE2010','POPESTIMATE2011', 'POPESTIMATE2012','POPESTIMATE2013',
                        'POPESTIMATE2014', 'POPESTIMATE2015']
    census_df_5 = census_df_5[columns_to_keep]
    df_CTYNAME = pd.Series()
    df_VALORMAX = pd.Series()
    cont = 0
    for i in range(0,6):
        for j in range(i+1,6):
            cont += 1
            name_1 = 'DIF1' + str(i) + '-1' + str(j)
            name_2 = 'POPESTIMATE201' + str(i)
            name_3 = 'POPESTIMATE201' + str(j)
            census_df_5[name_1] = abs(census_df_5[name_2] - census_df_5[name_3])
            valor_max = census_df_5[name_1].max()
            id_max = census_df_5[name_1].idxmax()
            df_CTYNAME[str(cont)] = census_df_5.loc[id_max]['CTYNAME']
            df_VALORMAX[str(cont)] = valor_max
    
    idmax_2 = df_VALORMAX.idxmax()
    #print(df_CTYNAME.loc[idmax_2])
    #print(census_df_5.head())
    return df_CTYNAME.loc[idmax_2]
answer_seven()

'Harris County'

### Question 8
In this datafile, the United States is broken up into four regions using the "REGION" column. 

Create a query that finds the counties that belong to regions 1 or 2, whose name starts with 'Washington', and whose POPESTIMATE2015 was greater than their POPESTIMATE 2014.

*This function should return a 5x2 DataFrame with the columns = ['STNAME', 'CTYNAME'] and the same index ID as the census_df (sorted ascending by index).*

In [40]:
def answer_eight():
    census_df_8 = census_df[census_df['REGION']!=4]
    census_df_8 = census_df_8[census_df_8['REGION']!=3]
    for x in census_df_8.index:
         if not census_df_8.loc[x]['CTYNAME'][:10]=='Washington':
                census_df_8 = census_df_8.drop(x)
         else:
            if not census_df_8.loc[x]['POPESTIMATE2015']>census_df_8.loc[x]['POPESTIMATE2014']:
                census_df_8 = census_df_8.drop(x)
    columns_to_keep = ['STNAME', 'CTYNAME']
    census_df_8 = census_df_8[columns_to_keep]
    #print(census_df_8)
    return census_df_8
answer_eight()

,STNAME,CTYNAME
896,Iowa,Washington County
1419,Minnesota,Washington County
2345,Pennsylvania,Washington County
2355,Rhode Island,Washington County
3163,Wisconsin,Washington County
